In [1]:
import pandas as pd
import os
import branca
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import folium as f
import geojson
from folium.features import GeoJsonPopup as pp
import mapclassify as mc

wgs84 = {'init':'EPSG:4326'}
bng = {'init':'EPSG:27700'}

In [2]:
# get the pharmacies in
fl = './../../data/food hazard/lines.geojson'
lines = gpd.read_file(fl)
lines2 = lines.to_json()

In [3]:
# get the pharmacies in
fl = './../../data/food hazard/points.geojson'
points = gpd.read_file(fl)

In [4]:
points = points.sort_values(['degree_centrality'], axis=0, ascending=False, inplace=False, kind='quicksort')
points['rank'] = np.arange(1, 151, 1)
points = points.sort_values(['name'], axis=0, ascending=True, inplace=False, kind='quicksort')
points

,name,x,y,degree,degree_centrality,out_degree,in_degree,geometry,rank
0,Afghanistan,66.004734,33.835231,6,0.040268,0.040268,0.040268,POINT (66.00473 33.83523),107
2,Albania,20.049834,41.142450,20,0.134228,0.134228,0.134228,POINT (20.04983 41.14245),83
35,Algeria,2.617323,28.158938,18,0.120805,0.120805,0.120805,POINT (2.61732 28.15894),85
3,Andorra,1.560544,42.542291,1,0.006711,0.006711,0.006711,POINT (1.56054 42.54229),139
1,Angola,17.537368,-12.293361,1,0.006711,0.006711,0.006711,POINT (17.53737 -12.29336),148
...,...,...,...,...,...,...,...,...,...
140,Uzbekistan,63.140015,41.755542,35,0.234899,0.234899,0.234899,POINT (63.14002 41.75554),75
141,Venezuela,-66.181841,7.124224,55,0.369128,0.369128,0.369128,POINT (-66.18184 7.12422),59
142,Vietnam,106.299147,16.646017,440,2.953020,2.953020,2.953020,POINT (106.29915 16.64602),19
143,Yemen,47.586762,15.909280,1,0.006711,0.006711,0.006711,POINT (47.58676 15.90928),142


In [5]:
points2 = points.to_json()

In [6]:
points['degree_quartile'] = pd.qcut(points['degree'], 4, labels=False)

In [7]:
lines = lines.reset_index()
lines

,index,Country.of.Origin,Notified.by,number.of.links,x_x,y_x,x_y,y_y,geometry
0,0,Afghanistan,Denmark,1,66.004734,33.835231,10.028010,55.981253,"LINESTRING (66.00473 33.83523, 10.02801 55.98125)"
1,1,Afghanistan,Finland,1,66.004734,33.835231,26.274666,64.498846,"LINESTRING (66.00473 33.83523, 26.27467 64.49885)"
2,2,Afghanistan,Sweden,2,66.004734,33.835231,16.745580,62.779665,"LINESTRING (66.00473 33.83523, 16.74558 62.77967)"
3,3,Afghanistan,United Kingdom,2,66.004734,33.835231,-2.865632,54.123872,"LINESTRING (66.00473 33.83523, -2.86563 54.12387)"
4,4,Albania,Croatia,4,20.049834,41.142450,16.404129,45.080476,"LINESTRING (20.04983 41.14245, 16.40413 45.08048)"
...,...,...,...,...,...,...,...,...,...
1446,1446,Vietnam,United Kingdom,14,106.299147,16.646017,-2.865632,54.123872,"LINESTRING (106.29915 16.64602, -2.86563 54.12..."
1447,1447,Vietnam,United States,12,106.299147,16.646017,-112.461674,45.679547,"LINESTRING (106.29915 16.64602, -112.46167 45...."
1448,1448,Yemen,Australia,1,47.586762,15.909280,134.491000,-25.732887,"LINESTRING (47.58676 15.90928, 134.49100 -25.7..."
1449,1449,Zimbabwe,Netherlands,1,29.851441,-19.004204,5.281448,52.100790,"LINESTRING (29.85144 -19.00420, 5.28145 52.10079)"


In [8]:
lines['number.of.links']

0        1
1        1
2        2
3        2
4        4
        ..
1446    14
1447    12
1448     1
1449     1
1450     1
Name: number.of.links, Length: 1451, dtype: int64

In [9]:
len(lines['number.of.links'].unique())

82

In [10]:
q = mc.Quantiles(lines['number.of.links'], k=15)


C:\Users\tk18583\AppData\Local\Continuum\anaconda3\envs\geo2\lib\site-packages\mapclassify\classifiers.py:235: UserWarning: Warning: Not enough unique values in array to form k classes
  Warn(
C:\Users\tk18583\AppData\Local\Continuum\anaconda3\envs\geo2\lib\site-packages\mapclassify\classifiers.py:238: UserWarning: Warning: setting k to 9
  Warn("Warning: setting k to %d" % k_q, UserWarning)


In [11]:
branca.colormap.linear.YlOrRd_09.scale(0, 50e3)

NameError: name 'branca' is not defined

In [ ]:
x = lines.get(lines['number.of.links'])


In [ ]:
df = pd.read_csv(county_data, na_values=[' '])

colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 50e3)
employed_series = df.set_index('FIPS_Code')['Employed_2011']

In [12]:
df

NameError: name 'df' is not defined

In [13]:
employed = employed_series.get(int(employed_series['id'][-5:]), None)
colorscale(employed)

NameError: name 'employed_series' is not defined

In [14]:
import branca
import pandas as pd
import os
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import requests
import folium as f
import geojson
import mapclassify as mc

url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
county_data = f'{url}/us_county_data.csv'
county_geo = f'{url}/us_counties_20m_topo.json'


# read the data
df = pd.read_csv(county_data, na_values=[' '])

# define the quantiles, teh vector is then sored as quantiles.yb
quantiles = mc.Quantiles(df['Unemployed_2011'], k=5)

colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 50e3)
employed_series = df.set_index('FIPS_Code')['Employed_2011']


def style_function(feature):
    employed = employed_series.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': quantiles.yb,
        'weight': 0,
        'fillColor': '#black' if employed is None else colorscale(employed)
    }


m = f.Map(
    location=[48, -102],
    tiles='cartodbpositron',
    zoom_start=3
)

f.TopoJson(
    json.loads(requests.get(county_geo).text),
    'objects.us_counties_20m',
    style_function=style_function
).add_to(m)


m

TypeError: Object of type ndarray is not JSON serializable

In [15]:
def ordinal(n):
    if n == 1:
        return "st"
    elif n == 2:
        return "nd"
    elif n == 3:
        return "rd"
    else: 
        return "th"

In [16]:
ordinal(5)

'th'

In [17]:
m=f.Map(location=[50, 2], zoom_start=2 , tiles='cartodbpositron')

colordict = {0:'#374545' , 1: '#2f5565' , 2: '#24749f', 3: '#1892d9', 4: '#1892d9'}

highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
style_function= lambda feature: {
                'opacity': 2,
                'weight': 0.2,
                'color': "#2F4F4F"
            }
f.features.GeoJson(lines, 
                   control = True, name = 'Food Hazards',
                  style_function=style_function,
                  highlight_function=highlight_function).add_to(m)




    
f.LayerControl().add_to(m)

#f.Choropleth(
#    lines[lines.geometry.length>0.001],
#    line_weight=1,
#   line_color='blue',
#    style_function=style_function
#).add_to(m)
 

    
    
#f.GeoJson(lines,name='lines').add_to(m)


In [21]:

for x, y, name, degree, in_degree, out_degree, degree_quartile, rank in zip(points['x'], points['y'], points['name'], points['degree'], points['in_degree'],points['out_degree'], points['degree_quartile'], points['rank']):
    f.CircleMarker(
        [y, x],
        radius=(degree/100)+10,
        popup = #f"Route {rank} to {ordinal(rank)}",
        f.Popup(f"<b>{str(name)}</b> <br> <br> Ranked <b>{str(rank)}</b>{str(ordinal(rank))} important" , max_width = 700),
        color='b',
        key_on = degree_quartile,
        threshold_scale=[0,1,2,3],
        fill_color=colordict[degree_quartile],
        fill=True,
        fill_opacity=0.8, control = True, name = 'Countries'
        ).add_to(m)

In [22]:
m

In [23]:
m.save('index.html')

In [ ]:
2468